In [43]:

import inspect
from typing import List, Union, Optional, Tuple, Any, Dict
import itertools


In [2]:
"""Generate contact matrices for different actions (quarantine levels)."""

from pathlib import Path
from typing import List

import pandas as pd


class ContactMatrix:
    """Contact matrix generator"""

    def __init__(self, **kwargs: Any) -> None:
        self._filename = "Social_contact_matrices_Philippines.xlsx"
        self._school = self._read_data("School")
        self._work = self._read_data("Work")
        self._other_location = self._read_data("other_location")
        self._home = self._read_data("Home")
        self._cm = self._get_contact(**kwargs)

    def _get_path(self, root: Path) -> Path:
        return Path.joinpath(root, 'data/contacts', self._filename)

    def _read_data(self, sheet_name: str = 'All_location') -> pd.DataFrame:

        def reader(path: Path) -> pd.DataFrame:
            return pd.read_excel(
                str(path),
                sheet_name,
                index_col="Ages"
            )

        try:
            return reader(self._get_path(Path.cwd()))
        except FileNotFoundError:
            return reader(self._get_path(Path.cwd().parent))

    def _return_zero(self, df: pd.DataFrame, mode: List[str]) -> pd.DataFrame:
        _df = df.copy(deep=True)
        _df[mode] = 0
        _df.loc[mode] = 0
        return _df

    def _compute_contact(self, df: pd.DataFrame, prop: float, sd: float) -> pd.DataFrame:
        return (prop - (prop * sd)) * df

    def _get_contact(
        self,
        home: bool = True,
        prop_work: float = 1.0,
        prop_school: float = 1.0,
        prop_other: float = 1.0,
        sd_work: float = 0.0,
        sd_school: float = 0.0,
        sd_other: float = 0.0,
        home_quarantine: list = [],
        not_allowed_work: list = [],
        not_allowed_other: list = [],
        not_allowed_school: list = []
    ):

        if home_quarantine:
            self._school, self._work, self._other_location = (
                self._return_zero(self._school, home_quarantine),
                self._return_zero(self._work, home_quarantine),
                self._return_zero(self._other_location, home_quarantine)
            )

        if not_allowed_work:
            self._work = self._return_zero(self._work, not_allowed_work)

        if not_allowed_other:
            self._other_location = self._return_zero(self._other_location, not_allowed_other)

        if not_allowed_school:
            self._school = self._return_zero(self._school, not_allowed_school)

        contact_school, contact_work, contact_other = (
            self._compute_contact(self._school, prop_school, sd_school),
            self._compute_contact(self._work, prop_work, sd_work),
            self._compute_contact(self._other_location, prop_other, sd_other)
        )

        contact = contact_school + contact_work + contact_other

        if home:
            contact += self._home

        return contact
    
    @property
    def cm(self) -> pd.DataFrame:
        return self._cm
    
    def _mean_cm(self, first_group: List[str], second_group: List[str]) -> float:
        return self.cm[first_group].loc[second_group].mean().mean()
    
    def _get_varname(self, variable: Any) -> str:
        local_variables = inspect.currentframe().f_back.f_locals.items()
        return [name for name, value in local_variables if value is variable][0]
    
    def get_cm(self, groups: Dict[str, List[str]]) -> Dict[str, float]:
        pass
    

    def get_cm(self, groups: Dict[str, List[str]]) -> Dict[str, float]:
        means = dict()
        for group_name, group_indices in groups.items():
            means[group_name] = self._mean_cm(group_indices, group_indices)

        return means

In [3]:
class AgeGroup:
    
    def _increment(self, ages: str) -> str:
        left, right = ages.split(sep='-')
        new_left, new_right = int(left) + 5, int(right) + 5
        return f'{new_left}-{new_right}'

    def _form_group(self, first) -> str:
        last = int(first) + 5
        return f'{first}-{last}'

    def ages(self, first, last) -> List[str]:

        group = self._form_group(first)
        last_index = group.split(sep='-')[1]
        groups = [group]

        while int(last_index) != last:
            groups.append(self._increment(group))
            group = self._increment(group)
            last_index = group.split(sep='-')[1]
        
        return groups

In [4]:
contact_matrix = ContactMatrix()

In [5]:
# Change this to dict

cm = AgeGroup()
children = cm.ages(0, 5)
adolescents = cm.ages(5, 20)
adults = cm.ages(20, 65)
elderly = cm.ages(65, 80)

In [6]:
cm_dict = {
    'children': children,
    'adolescents': adolescents, 
    'adults': adults, 
    'elderly': elderly
}

In [64]:
age_group_combinations = list(itertools.product(cm_dict.keys(), repeat=2))
_cm = {combination[0]: list() for combination in age_group_combinations}

for combination in age_group_combinations:
    _cm[combination[0]].append(contact_matrix._mean_cm(cm_dict[combination[0]], cm_dict[combination[1]]))

pd.DataFrame(_cm, index=_cm.keys())


,children,adolescents,adults,elderly
children,2.867183,1.180348,0.714234,0.069452
adolescents,0.928001,6.015802,0.725684,0.052568
adults,0.630153,1.027571,1.056809,0.041389
elderly,0.204199,0.344852,0.252879,0.105265


In [60]:
test = {combination[0]: list() for combination in age_group_combinations}